In [67]:
from pyquery import PyQuery as pq

import csv
import datetime
from glob import glob

In [77]:
rssdir = "rss/"
masterlist = []
for rssfile in glob(rssdir + "*"):
    indexfile = rssfile.replace(rssdir, "").replace(rssdir.replace('/', '\\'), '')
    with open(rssfile, "rb") as infile:
        html = infile.read()
        for item in pq(html)("item"):
            line = {}
            line["rssindex"] = indexfile
            authors = []
            for au in pq(item)("dc\:creator"):
                authors.append(pq(au).text().strip())
            line["authors"] = "|".join(authors)
            line['title'] = pq(pq(item)("title")).text().strip()
            line['introtext'] = pq(pq(pq(item)("description"))[0]).html()
            line['good-date'] = ""
            line['original-date'] = pq(pq(item).html().replace("pubDate", "pubdate"))("pubdate").text()
            line["good-date"] = datetime.datetime.strptime(line['original-date'], "%a, %d %b %Y %H:%M:%S %z")
            line["good-date"] = datetime.datetime.strftime(line['good-date'], "%Y-%m-%d %H%M%S%z")    
            line["maybe-archive"] = "https://web.archive.org/web/*/"
            line["maybe-archive"] += pq(pq(item)("link")).text().replace("https://cms.", "").replace("https://www.", "")    
            line['original-link'] = pq(pq(item)("link")).text()
            categories = []
            for cat in pq(item)("category"):
                categories.append(pq(cat).text())
            line['categories'] = '|'.join(categories)
            masterlist.append(line)

In [79]:
with open("the-messenger-rss-index.csv", "w", newline="", encoding="utf-8") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(list(masterlist[0].keys()))
    for row in sorted(masterlist, key=lambda d: d['good-date'], reverse=True):
        writer.writerow(list(row.values()))